# Image Super Resolution

In [ ]:
!pip install --upgrade pip

In [ ]:
!git clone https://github.com/idealo/image-super-resolution

In [ ]:
!pip install -e ".[gpu]"

In [ ]:
!pip install opencv-python

In [ ]:
!apt-get update

In [ ]:
!apt-get install libgl1-mesa-glx ffmpeg

In [1]:
# Importing all necessary libraries 
import cv2 
import os 
import numpy as np
import glob
from tqdm import tqdm

In [2]:
def runnerISR(path_to_images,path_to_output):
    
    from ISR.models import RDN
    import numpy as np
    from PIL import Image

    files = []

    for r, _, f in os.walk(path_to_images):
        for file in f:
            if '.jpg' in file:
                files.append([os.path.join(r, file),file.replace(".jpg","")])
    print("")
    files.sort()
    files.sort(key=lambda x: int(x[1]))
    #-----------------------------------------------
    model = RDN(weights='noise-cancel')
    
    for file in tqdm(range(len(files))):

        path = files[file][0]
        name = files[file][1]

        img = Image.open(path)
        sr_img = model.predict(np.array(img))
        sr_img = Image.fromarray(sr_img)
        sr_img.save(path_to_output+name+'.jpg','JPEG')
        

In [3]:
def vid2img(file,output):
    # Read the video from specified path 
    cam = cv2.VideoCapture(file) 
    
    try:    
        # creating a folder named data 
        if not os.path.exists(output): 
            os.makedirs(output) 
    
    # if not created then raise error 
    except OSError: 
        print ('Error: Creating directory of data') 
    
    # frame 
    currentframe = 0
    
    while(True): 
        
        # reading from frame 
        ret,frame = cam.read() 
    
        if ret: 
            # if video is still left continue creating images 
            name = output + str(currentframe) + '.jpg'
            print ('Creating...' + name) 
    
            # writing the extracted images 
            cv2.imwrite(name, frame) 
    
            # increasing counter so that it will 
            # show how many frames are created 
            currentframe += 1
            os.system("clear")
        else: 
            break
    
    # Release all space and windows once done 
    cam.release() 
    cv2.destroyAllWindows() 

In [4]:
def img2video(read_path,save_path):
    files = []
    for r, _, f in os.walk(read_path):
        for file in f:
            if '.png' in file:
                files.append([os.path.join(r, file),file.replace(".png","")])
    print("")
    files.sort(key=lambda x: int(x[1]))
    img = cv2.imread(files[0][0])
    height, width, _ = img.shape
    size = (width,height)
    print("VidSize: ",str(size))
    print("")
    #-----------------------------------------------
    out = cv2.VideoWriter(save_path+"reconstucted.mp4",cv2.VideoWriter_fourcc(*'mp4v'), 60, size)

    for file in tqdm(range(len(files))):
        path = files[file][0]
        img = cv2.imread(path)
        out.write(img)
    out.release()




In [6]:
!mkdir input/ && mkdir frames/ && mkdir HDframes/ && mkdir output/

mkdir: cannot create directory ‘input/’: File exists


In [5]:
vid2img("input/input.mp4","frames/")

Creating...frames/0.jpg
Creating...frames/1.jpg
Creating...frames/2.jpg
Creating...frames/3.jpg
Creating...frames/4.jpg
Creating...frames/5.jpg
Creating...frames/6.jpg
Creating...frames/7.jpg
Creating...frames/8.jpg
Creating...frames/9.jpg
Creating...frames/10.jpg
Creating...frames/11.jpg
Creating...frames/12.jpg
Creating...frames/13.jpg
Creating...frames/14.jpg
Creating...frames/15.jpg
Creating...frames/16.jpg
Creating...frames/17.jpg
Creating...frames/18.jpg
Creating...frames/19.jpg
Creating...frames/20.jpg
Creating...frames/21.jpg
Creating...frames/22.jpg
Creating...frames/23.jpg
Creating...frames/24.jpg
Creating...frames/25.jpg
Creating...frames/26.jpg
Creating...frames/27.jpg
Creating...frames/28.jpg
Creating...frames/29.jpg
Creating...frames/30.jpg
Creating...frames/31.jpg
Creating...frames/32.jpg
Creating...frames/33.jpg
Creating...frames/34.jpg
Creating...frames/35.jpg
Creating...frames/36.jpg
Creating...frames/37.jpg
Creating...frames/38.jpg
Creating...frames/39.jpg
Creating..

In [ ]:
runnerISR("frames/","HDframes/")

  7%|▋         | 259/3911 [07:00<1:37:17,  1.60s/it]

In [ ]:
img2video("HDframes/",save_path="output/")